In [1]:
import pandas as pd
import numpy as np

In [2]:
data_frame = pd.read_parquet('./dataset/relevance_challenge_valid.parquet')

In [3]:
def softmax(x):
    # Compute the exponential values for each element in the input array
    exps = np.exp(x - np.max(x))

    # Compute the softmax values by dividing the exponential of each element by the sum of exponentials
    return exps / np.sum(exps)  

In [4]:
data_frame['probabilities'] = data_frame['raw_prediction'].apply(softmax)

In [5]:
data_frame['pred'] = data_frame['raw_prediction'].apply(lambda x: np.argmax(x))

In [8]:
x_values = []
y_values = []
counts = []
for j in range(3000):
    filtered_data = data_frame[data_frame['pred'] == j]
    if not filtered_data.empty:
        x_values.append(j)
        y_values.append(filtered_data['distance'].mean())
        counts.append(len(filtered_data['distance']))


In [9]:
mean_distance_pred = pd.DataFrame([x for x in zip(x_values,y_values,counts)])

mean_distance_pred.rename(columns={0: 'feature'}, inplace=True)
mean_distance_pred.rename(columns={1:'mean_distance'},inplace=True)
mean_distance_pred.rename(columns={2:'counts'},inplace=True)

mean_distance_pred_t = mean_distance_pred[mean_distance_pred['counts'] >= 100]
mean_distance_pred_t.describe()

,feature,mean_distance,counts
count,124.000000,124.000000,124.000000
mean,323.475806,3200.795064,273.645161
std,462.741626,1998.821039,248.174447
min,0.000000,493.514270,100.000000
25%,43.500000,1741.740018,129.750000
50%,139.000000,2797.943047,174.000000
75%,438.250000,4310.125920,311.250000
max,2347.000000,10564.948403,1388.000000


In [10]:
valid_pred = mean_distance_pred['feature'].to_list()

In [11]:
data_frame = data_frame[data_frame['pred'].isin(valid_pred)]

In [12]:
bins = [0, 0.1, 0.2, 0.3, 1.0]

probabilities = data_frame['probabilities'].values
probabilities

array([array([1.0701745e-05, 3.0589050e-03, 1.1691522e-05, ..., 2.6032150e-05,
              1.1091998e-05, 2.5502461e-05], dtype=float32)                   ,
       array([4.49231920e-05, 4.37965238e-04, 7.01508368e-04, ...,
              5.86358547e-05, 1.19782715e-04, 1.44674719e-04], dtype=float32),
       array([6.8960690e-05, 5.7571520e-05, 6.7080755e-04, ..., 1.5246231e-04,
              2.9068040e-05, 2.0424993e-04], dtype=float32)                   ,
       ...,
       array([7.9940692e-05, 1.2463288e-04, 5.2878973e-03, ..., 4.0360184e-05,
              6.9534231e-05, 3.8948891e-04], dtype=float32)                   ,
       array([1.7136848e-04, 7.1163510e-04, 1.3841296e-04, ..., 6.5322194e-05,
              2.1939288e-04, 3.6347017e-05], dtype=float32)                   ,
       array([1.6614320e-03, 5.0464903e-05, 1.1213181e-04, ..., 3.3844783e-05,
              4.2440792e-05, 3.4518613e-05], dtype=float32)                   ],
      dtype=object)

In [13]:
histograms = []
for row in probabilities:
    histograms.append(np.histogram(row, bins=bins)[0])

In [14]:
histograms = np.array(histograms)
histograms = np.delete(histograms, 0, 1)
histograms

array([[0, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [15]:
column_names = [f'{bins[i]}_{bins[i+1]}' for i in range(len(bins) - 1)]
histograms = histograms.T

In [16]:
data_frame[column_names[1]] = histograms[0]
data_frame[column_names[2]] = histograms[1]
data_frame[column_names[3]] = histograms[2]

In [17]:
data_frame['peaks'] = data_frame['0.1_0.2'] + data_frame['0.2_0.3'] + data_frame['0.3_1.0']
data_frame.sort_values('confidence', ascending=True, inplace=True)
data_frame['distance'].describe()

count    55350.000000
mean      3260.978350
std       4202.391081
min          0.751505
25%        378.965748
50%       1254.280157
75%       4598.103009
max      19791.642460
Name: distance, dtype: float64

In [19]:
data_frame['text'] = data_frame['text'].str.replace(r'@[\w]+','')
data_frame['text'] = data_frame['text'].str.replace(r'https?://\S+|www\.\S+','')

/tmp/ipykernel_40731/3900086980.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data_frame['text'] = data_frame['text'].str.replace(r'@[\w]+','')
/tmp/ipykernel_40731/3900086980.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data_frame['text'] = data_frame['text'].str.replace(r'https?://\S+|www\.\S+','')


In [20]:
data_frame['text_length'] = data_frame['text'].str.len()

In [2]:
from sklearn.linear_model import LinearRegression

In [22]:
data_set = data_frame[['confidence','peaks','text_length','distance']]
data_set['features'] = data_set[['confidence' ,'peaks','text_length']].values.tolist()
data_set['label'] = data_set['distance'].apply(lambda x: 1/x)

data_set.drop(['confidence','peaks','distance','text_length'], axis=1, inplace=True)

/tmp/ipykernel_40731/3902920575.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set['features'] = data_set[['confidence' ,'peaks','text_length']].values.tolist()
/tmp/ipykernel_40731/3902920575.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set['label'] = data_set['distance'].apply(lambda x: 1/x)
/tmp/ipykernel_40731/3902920575.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/p

In [1]:
model = LinearRegression()
model.fit(data_frame['confidence','peaks','text_length'], data_set['label'])

NameError: name 'LinearRegression' is not defined

In [34]:
data_frame['score'] = data_frame['confidence'] * model.coef_[0] + data_frame['peaks'] * model.coef_[1] + data_frame['text_length'] * model.coef_[2] + model.intercept_
data_frame.sort_values('score', ascending=False, inplace=True)
data_frame_filtered = data_frame.iloc[0:int(0.1*len(data_frame))]
data_frame_filtered['distance'].describe()

count     5535.000000
mean       646.513765
std       1966.730383
min          2.974029
25%         32.053083
50%         82.422675
75%        296.950971
max      17147.333877
Name: distance, dtype: float64

[0.03665255946669803, 0.0014804699484368363, 3.4447546419646025e-06]

In [27]:
# lets try gradient descent

In [36]:
from scipy.optimize import minimize

# Define the objective function to minimize
def objective_function(weights):

    data_frame['score'] = weights[0] * data_frame['confidence'] + weights[1] * data_frame['peaks'] + weights[2] * data_frame['text_length'] +weights[3]

    print(data_frame['score'].mean())
    
    data_frame.sort_values('score', ascending=False, inplace=True)
    error = data_frame['distance'].iloc[:int(0.1*len(data_frame))].mean()
    return error

initial_weights = [1,1,1,1]

result = minimize(objective_function, initial_weights, method='nelder-mead', options={'xtol': 1e-8, 'disp': True})

optimal_weights = result.x

print("Optimal Weights:", optimal_weights)

/tmp/ipykernel_40731/4088886462.py:16: OptimizeWarning: Unknown solver options: xtol
  result = minimize(objective_function, initial_weights, method='nelder-mead', options={'xtol': 1e-8, 'disp': True})


65.23558585310592
65.23994870493013
65.2541621855359
68.424425961507
65.28558585310591
62.083215336831934
58.91261002449441
62.035567530927274
60.45769387119294
58.08077993493012
54.507739827666455
49.143816814946724
51.26188279185484
48.241850911920096
45.699300336677915
38.26081540320538
27.93491809256087
42.59193396896334
43.36877556089199
44.92245874474927
43.7571963568563
38.53936745375379
38.08838450224048
39.59840044220786
35.65184682672643
30.508857995432937
28.22911068199653
22.622999344604576
14.66481529002997
15.01700420328373
12.41406613850262
3.3666702100374617
2.262593215959487
-10.720665517059036
-10.279376632905546
18.381344411194267
-0.7258029515389421
-5.232866321039743
9.954536572202862
-7.235816766699537
9.189657275847594
-2.9079776970500633
6.738908004889632
-3.3684730361737754
6.050124697842251
-1.2621154187395025
-4.2294521699829986
3.4802304808859392
-1.6595579530266866
2.1952833724077827
-2.131691100993049
1.992079882279834
4.124192178293582
0.08446148051876919

In [37]:
model.coef_.tolist() + [model.intercept_]

[0.03665255946669803,
 0.0014804699484368363,
 3.4447546419646025e-06,
 0.001448278803293442]

In [38]:
import pickle

# Export the trained model
with open('trained_model.pkl', 'wb') as f:
    pickle.dump(model, f)


In [ ]:
import pickle

# Import the trained model
with open('trained_model.pkl', 'rb') as f:
    trained_model = pickle.load(f)


In [3]:
import pandas as pd

In [6]:
idx = pd.read_csv('./submission.csv')

In [7]:
dataset = pd.read_parquet('./dataset/relevance_challenge_test.parquet')

In [8]:
dataset.head()

,text,raw_prediction,confidence
49340,pq o bill não foi no 17🤯,"[1.8367281, -2.1598325, -0.68295133, -1.548769...",0.029528
313710,relax! @ clear light new mexico https://t.co/j...,"[-0.37366188, 0.6440394, 1.1217443, 0.93759936...",0.496777
406184,just wait until @gablesteveson becomes wwe cha...,"[-2.6163144, -2.9765346, -0.35192207, -0.39941...",0.004565
294082,@itsmainakageni this should not be a tbt maina...,"[-1.2409436, 0.34476605, -0.3286421, -1.021066...",0.402230
331979,@sandraeckersley @joneshowdareyou well he is a...,"[-0.6044651, -1.6236467, -0.14375791, 0.141036...",0.019685
